In [26]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import numpy as np
import lightgbm as lgbm
import os
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import time
import pickle

from tqdm import tqdm

from utils import load_json
from df_utils import form_last_quarter_df, form_pred_df
from features import QuarterlyFeatures, BaseCompanyFeatures, FeatureMerger
from targets import QuarterlyTarget
from data import load_quarterly_data_cf1, load_cf1_df, translate_currency_cf1
from models import GroupedOOFModel
import pipelines.marketcap
#from pipelines import marketcap
sns.set()

config = load_json("config.json")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
tickers_df = pd.read_csv('{}/cf1/tickers.csv'.format(config['data_path']))
tickers_df = tickers_df[tickers_df['currency']=='USD']
tickers_df = tickers_df[tickers_df['scalemarketcap'].apply(lambda x: x in ['4 - Mid', '5 - Large', '6 - Mega'])]
ticker_list = tickers_df['ticker'].unique().tolist()

In [9]:
columns = ['revenue', 'netinc', 'ncf', 'assets', 'ebitda', 'debt', 'fcf',
           'gp', 'workingcapital', 'cashneq', 'rnd', 'sgna', 'ncfx',
           'divyield', 'currentratio', 'netinccmn']

cat_columns = ['sector', 'sicindustry']

fc1 = QuarterlyFeatures(columns=columns,
                        quarter_counts=[2, 4, 10],
                        max_back_quarter=10)

fc2 = BaseCompanyFeatures(cat_columns=cat_columns)

feature = FeatureMerger(fc1, fc2, on='ticker')
target = QuarterlyTarget(col='marketcap', quarter_shift=0)
model = GroupedOOFModel(lgbm.sklearn.LGBMRegressor(), fold_cnt=5)
                
mc_pipeline = pipelines.marketcap.MarketcapPipeline(feature, target, model)
mc_pipeline.fit(config, ticker_list)
mc_pipeline.export_core()

3874it [00:30, 128.59it/s]
2396it [00:03, 685.28it/s]


0.3651129822271524


In [ ]:
0.3651129822271524

In [44]:
imp_df = pd.DataFrame()
imp_df['column'] = mc_pipeline.model.columns
imp_df['importance'] = mc_pipeline.model.base_models[0].feature_importances_
imp_df = imp_df.sort_values('importance', ascending=False)[:]
imp_df[:10]

,column,importance
480,sector,47
329,quarter10_revenue_diff_std,39
295,quarter4_divyield_diff_mean,38
130,quarter2_divyield_mean,37
407,quarter10_workingcapital_diff_max,36
455,quarter10_divyield_diff_mean,36
471,quarter10_netinccmn_median,36
481,sicsector,34
360,quarter10_ebitda_mean,31
380,quarter10_fcf_mean,30


In [11]:
pipeline_path = 'models_data/marketcap_pipeline_16.02.21_15:11'
mc_pipeline = pipelines.marketcap.load(pipeline_path)

In [13]:
mc_df = mc_pipeline.execute(config, ticker_list)

3874it [00:30, 128.52it/s]
